In [ ]:
'''패킹된 샘플 데이터셋 만들기'''
import os

files_path = "./Benign PE Samples UPX/"
files = os.listdir(files_path)
files_paths = [files_path + x for x in files]

# 디렉터리 B에 있는 각 파일에 대해 upx를 실행한다.
from subprocess import Popen, PIPE

cmd = 'upx.exe'
for path in files_paths:
    cmd2 = cmd + "\"" + path + "\""
    res = Popen(cmd2, stdout=PIPE).communicate()
    print(res)

# 패킹 시 에러가 발생할 때마다 원래의 샘플을 삭제한다.
if "error" in str(res[0]):
    print(path)
    os.remove(path)

In [ ]:
'''패커 분류기 구축'''
import os

# 사용된 패커에 해당하는 레이블과 함께, 분석해야 할 파일의 이름을 읽는다.
directories_with_labels = [
    ("Benign PE Samples", 0),
    ("Benign PE Samples UPX", 1),
    ("Benign PE Samples Amber", 2),
]
list_of_samples = []
labels = []
for dataset_path, label in directories_with_labels:
    samples = [f for f in os.listdir(dataset_path)]
    for file in samples:
        file_path = os.path.join(dataset_path, file)
        list_of_samples.append(file_path)
        labels.append(label)

# 훈련 데이터와 테스트 데이터를 분할한다.
from sklearn.model_selection import train_test_split

samples_train, samples_test, labels_train, labels_test = train_test_split(
    list_of_samples, labels, test_size=0.3, stratify=labels, random_state=11
)

# N-그램 추출에 사용할 함수
import collections
from nltk import ngrams
import numpy as np

def read_file(file_path):
    # 이진 파일의 문자열을 읽는다.
    with open(file_path, 'rb') as binary_file:
        data = binary_file.read()
    return data

def byte_sequence_to_Ngrams(byte_sequence, N):
    # 바이트 문자열에서 N-그램 리스트를 만든다.
    Ngrams = ngrams(byte_sequence, N)
    return list(Ngrams)

def extract_Ngrams_counts(file, N):
    # 이진 파일을 읽고, 이진 문자열에서 N-그램의 갯수를 출력한다.
    filebyte_sequence = read_file(file)
    file_Ngrams = byte_sequence_to_Ngrams(filebyte_sequence, N)
    return collections.Counter(file_Ngrams)

def featurize_sample(sample, K1_most_frequent_Ngrams_list):
    # 샘플의 특성 벡터를 만든다. 특성은 우리가 선택한 N-그램의 K1개의 갯수이다.
    K1 = len(K1_most_frequent_Ngrams_list)
    feature_vector = K1 * [0]
    file_Ngrams = extract_Ngrams_counts(sample, N)
    for i in range(K1):
        feature_vector[i] = file_Ngrams[K1_most_frequent_Ngrams_list[i]]
    return feature_vector

In [ ]:
# 데이터에 대해 특성으로 사용하려는 N-그램을 선택한다.
N = 2
total_Ngram_count = collections.Counter([])
for file in samples_train:
    total_Ngram_count += extract_Ngrams_counts(file, N)

K1 = 100
K1_most_common_Ngrams = total_Ngram_count.most_common(K1)
K1_most_common_Ngrams = [x[0] for x in K1_most_common_Ngrams]

In [ ]:
# 훈련데이터를 특성화한다.
Ngram_features_list_train = []
y_train = []
for i in range(len(samples_train)):
    file = samples_train[i]
    Ngram_features = featurize_sample(file, K1_most_common_Ngrams)
    Ngram_features_list_train.append(Ngram_features)
    y_train.append(labels_train[i])
X_train = Ngram_features_list_train

In [ ]:
# 훈련 데이터에 대해 랜덤 포레스트 모델을 훈련한다.
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)

In [ ]:
# 테스트 데이터를 특성화한다.
Ngram_features_list_test = []
y_test = []
for i in range(len(samples_test)):
    file = samples_test[i]
    Ngram_features = featurize_sample(file, K1_most_common_Ngrams)
    Ngram_features_list_train.append(Ngram_features)
    y_train.append(labels_test[i])
X_test = Ngram_features_list_test

In [ ]:
# 훈련한 분류기를 사용해 테스트 데이터를 예측하고, 혼동행렬을 사용해 성능을 평가한다.
y_pred = clf.predict(X_test)
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_pred))